In [2]:
import os
import ee
import collections
collections.Callable = collections.abc.Callable

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

#from google.colab import drive
#drive.mount('/content/gdrive')

#!pip install geemap
#import geemap

#os.chdir('/content/gdrive/MyDrive/SNiP/RS_fusion/Nietupski_etal_code/GEE-Image-Fusion-Bands-EF')
#! ls

Enter verification code: 4/1ARtbsJqvP_m-mrHSB_DjgLxDLfFwPHmaQcX3lzBHFTWeEiejmxqeON6-3cw

Successfully saved authorization token.


In [51]:
pip install geemap


     ---------------------------------------- 2.1/2.1 MB 3.3 MB/s eta 0:00:00
  Using cached PyCRS-1.0.2.tar.gz (36 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t


     ------------------------------------- 942.0/942.0 KB 14.8 MB/s eta 0:00:00
  Using cached geojson-2.5.0-py2.py3-none-any.whl (14 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 95.0/95.0 KB 1.8 MB/s eta 0:00:00
  Using cached whiteboxgui-0.7.0-py2.py3-none-any.whl (99 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
     ---------------------------------------- 1.3/1.3 MB 8.2 MB/s eta 0:00:00
     -------------------------------------- 198.1/198.1 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ipyfilechooser-0.6.0-py3-none-any.whl (11 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
     ---------------------------------------- 61.6/61.6 KB 1.7 MB/s eta 0:00:00
     -------------------------------------- 160.9/160.9 KB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 3.6/3.6 MB 16.3 MB/s eta 0:00:00
     ---------------------------------------- 71.8/71.8 KB 4.1 MB/s eta 0:00:00
     -------------------------------------- 106.8/106.8 KB 3.1 MB/s eta 0:00:00
  Created wheel for ee-extra: filename=ee_extra-0.0.14-py3-none-any.whl size=209114 sha256=1dbf5ffce73ba168e5b3c58209eedfae67d9e183421fad861a5bcf681166094b
  Stored in directory: c:\users\ellie\appdata\local\pip\cache\wheels\8f\82\90\5f797fbb4cead12904c9ccab32657b2b1a2d31c1d2f3bb869a
  Created wheel for sankee: filename=sankee-0.2.0-py3-none-any.whl size=28745 sha256=3f273af91c318c409487284f6ade038577f918bbb24d9850b72c89e74d305a69
  Stored in directory: c:\users\ellie\appdata\local\pip\cache\wheels\bc\d8\24\da019edf0a73f225309355eed7152751

In [86]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import geemap

In [ ]:
#!git clone https://github.com/tytupski/GEE-Image-Fusion.git

In [3]:
#Optional for if executing in colab cells
from GEE_ImageFusion_Fxns_81322.core_functions_81322 import *
#from GEE_ImageFusion_Fxns.copy_of_get_paired_collections import *
from GEE_ImageFusion_Fxns_81322.prep_functions_81322 import *

In [15]:
# -*- coding: utf-8 -*-


import ee

##############################################################################
# MASKING, INDEX CALCULATION, L5 & L7 TO L8 HARMONIZATION
##############################################################################

#Landsat8 SR scaling function. Applies scaling factors.
def applyScaleFactors(image):
    opticalBands = image.select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2']).multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True)\
                .multiply(10000)\
                .copyProperties(image, ['system:time_start', 'DOY', 'system:footprint', 'system:index'])

########
#Landsat
########
def fmask89(region):
    """
    Mask cloud, shadow, and snow with fmask and append the percent of pixels \
    masked as new image property.

    Parameters
    ----------
    image : image.Image
        Landsat 8 or 9 image with qa band.

    Returns
    -------
    image.image
        Masked landsat image with CloudSnowMaskedPercent property.

    """
    def allMasks89(img):
    # Bits 3 and 5 are cloud shadow and cloud, respectively. 4 is snow
        orig = img
        cirrusBitMask = 1 << 2
        cloudShadowBitMask = 1 << 3
        snowBitMask = 1 << 4
        cloudsBitMask = 1 << 5

        # Get the pixel QA band.
        qa = img.select('QA_PIXEL')

        # make mask
        mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
            .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
            .And(qa.bitwiseAnd(snowBitMask).eq(0)) \
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

        # select water pixels
        #water = gsw.select('occurrence').gte(40);
        #waterMask = water.selfMask()

        # mask the mask with the mask...
        maskedMask = mask.updateMask(mask) #.updateMask(waterMask)

        # count the number of nonMasked pixels
        maskedCount = maskedMask.select(['QA_PIXEL']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=region,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        # count the total number of pixels
        origCount = img.select(['blue']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=region,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        # calculate the percent of masked pixels
        percent = ee.Number(origCount.get('blue')) \
            .subtract(maskedCount.get('QA_PIXEL')) \
            .divide(origCount.get('blue')) \
            .multiply(100) \
            .round()

        # Return the masked image with new property and time stamp
        return img.updateMask(mask) \
            .set('CloudSnowMaskedPercent', percent) \
            .copyProperties(img, ['system:time_start', 'CLOUD_COVER','LANDSAT_SCENE_ID','SPACECRAFT_ID','WRS_PATH','WRS_ROW', 'system:index', 'system:footprint'])

    return allMasks89

#Mask cloud, cloud shadow, and snow in Landsat 5, 7 images
def fmask57(region):
    """
    Mask cloud and snow with fmask and append the percent of pixels \
    masked as new image property.

    Parameters
    ----------
    image : image.Image
        Landsat 5 or 7 image with qa band.

    Returns
    -------
    image.image
        Masked landsat image with CloudSnowMaskedPercent property.

    """
    def allMasks57(img):
    # Bits 3 and 5 are cloud shadow and cloud, respectively. 4 is snow
        orig = img
        cloudShadowBitMask = 1 << 5
        snowBitMask = 1 << 4
        cloudsBitMask = 1 << 3

        # Get the pixel QA band.
        qa = img.select('QA_PIXEL')

        # make mask
        mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
            .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
            .And(qa.bitwiseAnd(snowBitMask).eq(0)) \

        # select water pixels
        #water = gsw.select('occurrence').gte(40);
        #waterMask = water.selfMask()

        # mask the mask with the mask...
        maskedMask = mask.updateMask(mask) #.updateMask(waterMask)

        # count the number of nonMasked pixels
        maskedCount = maskedMask.select(['QA_PIXEL']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=region,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        # count the total number of pixels
        origCount = img.select(['blue']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=region,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        # calculate the percent of masked pixels
        percent = ee.Number(origCount.get('blue')) \
            .subtract(maskedCount.get('QA_PIXEL')) \
            .divide(origCount.get('blue')) \
            .multiply(100) \
            .round()

        # Return the masked image with new property and time stamp
        return img.updateMask(mask) \
            .set('CloudSnowMaskedPercent', percent) \
            .copyProperties(img, ['system:time_start', 'CLOUD_COVER','LANDSAT_SCENE_ID','SPACECRAFT_ID','WRS_PATH','WRS_ROW', 'system:index', 'system:footprint'])

    return allMasks57

#########
#Sentinel
#########

#Combine s2 and s2 cloudless collections
def get_s2_sr_cld_col(aoi, start_date, end_date, CLOUD_FILTER):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

#Add cloud band to combined S2 collection
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

#Add shadow band to combined S2 collection
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

#Assemble all cloud and cloud shadow components for final mask
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

#Define a funciton to apply the cloud mask to each image in the collection
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select(['B.*', 'QA60']).updateMask(not_cld_shdw)

# Function to mask clouds using the Sentinel-2 QA band
# @param {ee.Image} image Sentinel-2 image
# @return {ee.Image} cloud masked Sentinel-2 image
#UNSCALED RIGHT NOW

def maskS2clouds(region):
    def allMasksS2(img):

        qa = img.select('QA60')

        # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11

        # Both flags should be set to zero, indicating clear conditions.
        mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

        # mask the mask with the mask...
        maskedMask = mask.updateMask(mask)

        # count the number of nonMasked pixels
        maskedCount = maskedMask.select(['QA60'])\
            .reduceRegion(
                reducer = ee.Reducer.count(),\
                geometry = region,\
                scale = ee.Number(30),\
                maxPixels = ee.Number(4e10))

        # count the total number of pixels
        origCount = img.select(['blue'])\
            .reduceRegion(reducer = ee.Reducer.count(),\
                          geometry = region,\
                          scale = ee.Number(30),\
                          maxPixels = ee.Number(4e10))\

        # calculate the percent of masked pixels
        percent = ee.Number(origCount.get('blue'))\
            .subtract(maskedCount.get('QA60'))\
            .divide(origCount.get('blue'))\
            .multiply(100)\
            .round()\

        return img.updateMask(mask)\
            .set('CloudSnowMaskedPercent', percent)\
            .copyProperties(img, ['system:time_start', 'CLOUDY_PIXEL_PERCENTAGE','CLOUD_SHADOW_PERCENTAGE','MGRS_TILE','SNOW_ICE_PERCENTAGE','SPACECRAFT_NAME','WATER_PERCENTAGE', 'VEGETATION_PERCENTAGE', 'UNCLASSIFIED_PERCENTAGE', 'system:index','system:footprint']);

    return allMasksS2






######
#MODIS
######
def maskMODIS(image):
    """
    Mask snow covered and extremely high albedo areas from the modis images.

    Parameters
    ----------
    image : image.Image
        MODIS image.

    Returns
    -------
    image.image
        Masked MODIS image.

    """
    # calculate snow water index for the image
    swi = image.expression(
        '(green * (nir - swir1)) / ((green + nir) * (nir + swir1))',
        {'green': image.select(['green']),
         'nir': image.select(['nir']),
         'swir1': image.select(['swir1'])
         }).rename('swi')

    # mask out values of swi above 0.1
    mask = swi.lt(0.1)

    return image \
        .updateMask(mask) \
        .copyProperties(image, ['system:time_start'])

def maskLandsat(img):
    """
    Mask cloud, shadow, and snow with fmask and append the percent of pixels \
    masked as new image property.

    Parameters
    ----------
    image : image.Image
        Landsat 8 or 9 image with qa band.

    Returns
    -------
    image.image
        Masked landsat image with CloudSnowMaskedPercent property.

    """
    # Bits 3 and 5 are cloud shadow and cloud, respectively. 4 is snow
    cirrusBitMask = 1 << 2
    cloudShadowBitMask = 1 << 3
    snowBitMask = 1 << 4
    cloudsBitMask = 1 << 5

    # Get the pixel QA band.
    qa = img.select('QA_PIXEL')

    # make mask
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
        .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
        .And(qa.bitwiseAnd(snowBitMask).eq(0)) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # select water pixels
    #water = gsw.select('occurrence').gte(40);
    #waterMask = water.selfMask()

    # mask the mask with the mask...
    maskedMask = mask.updateMask(mask) #.updateMask(waterMask)

    # count the number of nonMasked pixels
    maskedCount = maskedMask.select(['QA_PIXEL']) \
        .reduceRegion(reducer=ee.Reducer.count(),
                      geometry=img.geometry(),
                      scale=ee.Number(30),
                      maxPixels=ee.Number(4e10))

    # count the total number of pixels
    origCount = img.select(['blue']) \
        .reduceRegion(reducer=ee.Reducer.count(),
                      geometry=img.geometry(),
                      scale=ee.Number(30),
                      maxPixels=ee.Number(4e10))

    # calculate the percent of masked pixels
    percent = ee.Number(origCount.get('blue')) \
        .subtract(maskedCount.get('QA_PIXEL')) \
        .divide(origCount.get('blue')) \
        .multiply(100) \
        .round()

    # Return the masked image with new property and time stamp
    return img.updateMask(mask) \
        .set('CloudSnowMaskedPercent', percent) \
        .copyProperties(img, ['system:time_start', 'SPACECRAFT_ID', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'WRS_ROW'])





# def addNDVI(image):
#     """
#     Mask snow covered and extremely high albedo areas from the modis images.

#     Parameters
#     ----------
#     image : image.Image
#         Landsat or MODIS image with bands named 'nir' and 'red'.

#     Returns
#     -------
#     image.image
#         Image with additional NDVI band.
#     """
#     # calculate NDVI
#     ndvi = image.normalizedDifference(['nir', 'red']).select(['nd'], ['ndvi'])

#     return image.addBands(ndvi)



##############################################################################
# FILTER AND PAIR IMAGES
##############################################################################


def getPaired(startDate, endDate, ls89, ls57, s2_sr_cloudless,
              landsatCollection, landsatBands, bandNamesLandsat,
              S2Bands, bandNamesS2,
              modisCollection, modisBands, bandNamesModis,
              commonBandNames,
              region):
    
    """
    Create a list of image collections. Landsat and MODIS with low cloud cover\
    from the same date and the MODIS images between these pairs.

    Parameters
    ----------
    startDate: str
        Start date of fusion timeframe.
    endDate: str
        End date of the fusion timeframe.
    landsatCollection: str
        Landsat collection https://developers.google.com/earth-engine/datasets
    landsatBands: ee_list.List
        List of integers corresponding to Landsat bands.
    bandNamesLandsat: ee_list.List
        List of strings used to rename bands.
    modisCollection: str
        MODIS collection https://developers.google.com/earth-engine/datasets
    modisBands: ee_list.List
        List of integers corresponding to MODIS bands in same order as Landsat.
    bandNamesModis: ee_list.List
        List of strings used to rename bands.
    commonBandNames: ee_list.List
        List of bands to use in fusion. Common to both Landsat and MODIS.
    region: geometry.Geometry
        Location to use in filtering collections. Must not be in scene overlap.

    Returns
    -------
    python list obejct
        Each element in this list is an ee.ImageCollection. The first and \
        second elements are the Landsat occuring on the same date and \
        the last element is the MODIS images between each of the pair \
        dates.

        """
    
    if startDate >= '2013-03-18':
        col89 = ee.ImageCollection(ls89) \
                    .filterDate(startDate, endDate) \
                    .filterBounds(region) \
                    .select(landsatBandsStr, bandNamesLandsat) \
                    .map(fmask89(region)) \
                    .filterMetadata('CloudSnowMaskedPercent', 'less_than', 50)\
                    .map(lambda image: image \
                         .setMulti({
                             'system:time_start':
                                 ee.Date(image.date().format('y-M-d')) \
                                 .millis(),
                             'DOY': image.date().format('D')
                             })) \
                    .select(commonBandNames) \
                    .sort('system:time_start')

        s2 = ee.ImageCollection(s2_sr_cloudless)\
            .map(lambda image: image.clip(col89.first().geometry())) \
            .map(add_cld_shdw_mask)\
            .map(apply_cld_shdw_mask)\
            .select(S2Bands, bandNamesS2)\
            .map(maskS2clouds(region))\
            .filter(ee.Filter.lte('CloudSnowMaskedPercent', 50))\
            .map(lambda img: img.resample('bicubic')\
                .setMulti({
                     'system:time_start':
                         ee.Date(img.date().format('y-M-d')) \
                         .millis(),
                     'DOY': img.date().format('D')
                     }))\
            .map(lambda img: img.reproject(**{'crs': ee.String(col89.first().projection().crs()), 'scale': ee.Number(30)}))\
            .select(commonBandNames)


        fine = col89.merge(s2).sort('system:time_start')
        
        #For some tiles, Sentinel images from 2A and 2B can occur on the same day, or concurrent with Landsat
        #which must be filtered out and if taken near the same time, act as validation
        fineDistinct = fine.distinct('system:time_start');

        print('ORIG Fine # images', fineDistinct.size().getInfo())    

        #Define an equals filter to match collections
        dateFilter = ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
              })

        #Define inverted join to separate and save only the duplicates
        invertedJoin = ee.Join.inverted()  

        #Apply the join
        fineDup = ee.ImageCollection(invertedJoin.apply(fine, fineDistinct, dateFilter))

    else: 
        col57 = ee.ImageCollection(ls57)\
            .filterDate(startDate, endDate)\
            .filterBounds(region)\
            .select(landsatBands57, bandNamesLandsat)\
            .map(fmask57(region))\
            .filter(ee.Filter.lte('CloudSnowMaskedPercent', 50))\
            .map(lambda image: image \
                 .setMulti({
                     'system:time_start':
                         ee.Date(image.date().format('y-M-d')) \
                         .millis(),
                     'DOY': image.date().format('D')
                     })) \
            .select(commonBandNames) \
            .sort('system:time_start')\

        fine = col57    

    

   
    # get modis images
    modis = ee.ImageCollection(modisCollection) \
              .filterDate(startDate, endDate) \
              .filterBounds(fine.first().geometry()) \
              .map(lambda image: image.clip(fine.first().geometry())) \
              .select(modisBands, bandNamesModis) \
              .map(maskMODIS) \
              .map(lambda image: image.set('DOY', image.date().format('D'))) \
              .select(commonBandNames)\
              .sort('system:time_start')\

    
    print('ORIG MODIS # images', modis.size().getInfo())

    # filter the two collections by the date property
    dayfilter = ee.Filter.equals(leftField='system:time_start',
                                 rightField='system:time_start')

    # define simple join
    pairedJoin = ee.Join.simple()
    # define inverted join to find modis images without landsat pair
    invertedJoin = ee.Join.inverted()

    # create collections of paired landsat and modis images
    landsatPaired = pairedJoin.apply(fineDistinct, modis, dayfilter)
    modisPaired = pairedJoin.apply(modis, fineDistinct, dayfilter)
    
    #IF ONLY USING LANDSAT 5, 7: 
    #modisUnpaired = invertedJoin.apply(modis, fineDistinct, dayfilter)
    
    #IF USING SENTINEL
    
    #Filter collection for non-consecutive paired landsat/modis images and store as a validation collection
    def dateDiffProp(img):
        index = lst.indexOf(img)
        img = ee.Image(img)
        imgDate = img.date().format('yyyy-MM-dd')
        previousIndex = ee.Algorithms.If(index.eq(0), index, index.subtract(1))
        prevImgDate = ee.Image(lst.get(previousIndex)).date().format('yyyy-MM-dd')
        diff = ee.Number(ee.Date(imgDate).difference(ee.Date(prevImgDate), 'day'))
        diffProp = ee.Image(img).set('DateDiff', diff)
        return diffProp
    
    sortImgLength_0 = landsatPaired.size()
    lst = landsatPaired.toList(sortImgLength_0)
    dateProp_0 = ee.ImageCollection(lst.map(dateDiffProp))

    fineImgCons = dateProp_0.filter(ee.Filter.eq('DateDiff', 1))
    landsatPaired = dateProp_0.filter(ee.Filter.neq('DateDiff', 1))

    
    sortImgLength_1 = modisPaired.size()
    lst = modisPaired.toList(sortImgLength_1)
    dateProp_1 = ee.ImageCollection(lst.map(dateDiffProp))
  
    coarseImgCons = dateProp_1.filter(ee.Filter.eq('DateDiff', 1))
    modisPaired = dateProp_1.filter(ee.Filter.neq('DateDiff', 1))


    #Merge duplicates (fine) and consecutives (fine and coarse) to one image collection
    valImgs = fineImgCons.merge(coarseImgCons).merge(fineDup)

  
    #Create collection of unpaired modis images
    #Use the list of distinct, non-consecutive fine images
    fineDistinctLength = fineDistinct.size()
    lst = fineDistinct.toList(fineDistinctLength)
    fineDist_dateProp = ee.ImageCollection(lst.map(dateDiffProp))

    fineDist_dateProp = fineDist_dateProp.filter(ee.Filter.neq('DateDiff', 1))
  
    #Create collection of unpaired modis images between the fine/coarse pairs
    modisUnpaired = invertedJoin.apply(modis, fineDist_dateProp, dayfilter)

    
    sortedImages = [landsatPaired, modisPaired, modisUnpaired]
    
    return sortedImages, valImgs


In [16]:

##############################################################################
# CREATE SUBCOLLECTIONS FOR EACH SET OF LANDSAT/MODIS PAIRS
##############################################################################


def getDates(image, empty_list):
    """
    Get date from image and append to list.

    Parameters
    ----------
    image : image.Image
        Any earth engine image.
    empty_list : ee_list.List
        Earth engine list object to append date to.

    Returns
    -------
    updatelist : ee_list.List
        List with date appended to the end.

    """
    # get date and update format
    date = ee.Image(image).date().format('yyyy-MM-dd')

    # add date to 'empty list'
    updatelist = ee.List(empty_list).add(date)

    return updatelist


def makeSubcollections(paired):
    """
    Reorganize the list of collections into a list of lists of lists. Each\
    list within the list will contain 3 lists. The first of these three will\
    have the earliest and latest Landsat images. The second list will have the\
    earliest and latest MODIS images. The third list will have all the MODIS\
    images between the earliest and latest pairs.\
    (e.g. L8 on 05/22/2017 & 06/23/2017, MOD 05/23/2017 & 06/23/2017,\
     MOD 05/23/2017 through 06/22/2017).

    Parameters
    ----------
    paired : python List
        List of image collections. 1. Landsat pairs, 2. MODIS pairs, and\
        3. MODIS between each of the pairs.

    Returns
    -------
    ee_list.List
        List of lists of lists.

    """
    
    def updateDates(ind):
        newDateList = ee.List([])
        date1 = ee.List(dateList).get(ee.Number(ind))
        date2 = ee.Date(ee.List(dateList).get(ee.Number(ind).add(1)))
        date3 = ee.Date(ee.List(dateList).get(ee.Number(ind).add(2)))
    
        
        diff = date2.difference(date1, 'day')
        diff2 = date3.difference(date1, 'day')

        
        datesUpdate = ee.Algorithms.If(ee.Number(diff).eq(ee.Number(1)),\
                                       ee.List(newDateList).add(1),\
                                       dateList)
        return datesUpdate
 
    
    def getSub(ind):
        """
        Local function to create individual subcollection.

        Parameters
        ----------
        ind : int
            Element of the list to grab.

        Returns
        -------
        ee_list.List
            List of pairs lists for prediction 2 pairs and images between.

        """
        date1 = ee.List(dateList).get(ee.Number(ind))
        date2 = ee.Date(ee.List(dateList).get(ee.Number(ind).add(1)))
        date3 = ee.Date(ee.List(dateList).get(ee.Number(ind).add(2)))
    
        
        diff = date2.difference(date1, 'day')
        diff2 = date3.difference(date1, 'day')

        
        datesUpdate = ee.Algorithms.If(ee.Number(diff).eq(ee.Number(1)),\
                                       ee.List(dateList).remove((ee.List(dateList).get(ee.Number(ind).add(1)))),\
                                       dateList)
        
        # get landsat images
        lan_01 = sortedImages[0] \
            .filterDate(ee.List(datesUpdate).get(ind),
                        ee.Date(ee.List(datesUpdate).get(ee.Number(ind).add(1)))\
                            .advance(1, 'day')) \
            .toList(2)
        # get modis paired images
        mod_01 = sortedImages[1] \
            .filterDate(ee.List(datesUpdate).get(ind),
                        ee.Date(ee.List(datesUpdate).get(ee.Number(ind).add(1)))\
                            .advance(1, 'day')) \
            .toList(2)
        # get modis images between these two dates
        mod_p = sortedImages[2] \
            .filterDate(ee.List(datesUpdate).get(ind),
                        ee.Date(ee.List(datesUpdate).get(ee.Number(ind).add(1)))\
                            .advance(1, 'day'))

        mod_p = mod_p.toList(mod_p.size())

        # combine collections to one object
        subcollection = ee.List([lan_01, mod_01, mod_p])

        return subcollection

    # empty list to store dates
    empty_list = ee.List([])

    # fill empty list with dates
    dateList = paired[0].iterate(getDates, empty_list)
    print(ee.List(dateList).getInfo())
    
    dateList2 = ee.List.sequence(0, ee.List(dateList).length().subtract(2))\
        .map(updateDates)
    print(dateList2.getInfo())

    # filter out sub collections from paired and unpaired collections
    subcols = ee.List.sequence(0, ee.List(dateList).length().subtract(2))\
        .map(getSub)

    return subcols.distinct()



#####
#new export functions
########

##OPTION 1
#For points:
#Buffer and water mask for gauge site points
#Mask for water and buffer as per Aquasat

#occ = gsw.select('occurrence')

def waterfunc(buf):
    #Define a 200m buffer around each point
    invBuf = buf.buffer(200).geometry()
    #Clip the pekel mask to this buffer
    pekclip = occ.clip(invBuf)
    #Reduce the buffer to pekel min and max
    pekMin = pekclip.reduceRegion(ee.Reducer.minMax(), invBuf, 30)
    #Add another reducer to get the median pekel occurnce
    pekMed = pekclip.reduceRegion(ee.Reducer.median(),invBuf,30)
    #Define the output features
    waterBuf = buf.set({'max':pekMin.get('occurrence_max')})\
          .set({'min':pekMin.get('occurrence_min')})\
          .set({'med':pekMed.get('occurrence')})

    return waterBuf

##OPTION 2
#For reaches
#fxn to get the geometry bounds of each feature in the collection
def getBounds(feature):
  #keep list of properties
  keepProperties = ['REACH_ID', 'LENGTH_KM']
  #get the bounds
  bounds = feature.geometry().bounds()
  #Return new Feature, copying old feature properties
  return ee.Feature(bounds).copyProperties(feature, keepProperties)


#Zonal stats fxn MEAN
def zonalStatsMean(ic, fc, params):
#Initialize internal params dictionary.
    _params = {
        'reducer': ee.Reducer.mean(),
        'scale': None,
        'crs': None,
        'bands': None,
        'bandsRename': None,
        'imgProps': None,
        'imgPropsRename': None,
        'datetimeName': 'datetime',
        'datetimeFormat': 'YYYY-MM-dd HH:mm:ss'
    }

  #Replace initialized params with provided params.
    if params:
        for param in params:
            _params[param] = params[param] or _params[param]

    #print(params)

  #Set default parameters based on an image representative.
    imgRep = ic.first()
    nonSystemImgProps = ee.Feature(None) \
        .copyProperties(imgRep).propertyNames()

    if not _params['bands']: \
     _params['bands'] = imgRep.bandNames() \

    if not _params['bandsRename']: \
      _params['bandsRename'] = _params['bands'] \

    if not _params['imgProps']: \
       _params['imgProps'] = nonSystemImgProps \

    if not _params['imgPropsRename']: \
       _params['imgPropsRename'] = _params['imgProps']
  
    def reducing(img):
    #Select bands (optionally rename), set a datetime & timestamp property.
        img = ee.Image(img.select(_params['bands'], _params['bandsRename'])) \
            .set(_params['datetimeName'], img.date().format(_params['datetimeFormat'])) \
            .set('timestamp', img.get('system:time_start'))

    #Define final image property dictionary to set in output features.
        propsFrom = ee.List(_params['imgProps']) \
            .cat(ee.List([_params['datetimeName'], 'timestamp']))

        propsTo = ee.List(_params['imgPropsRename']) \
            .cat(ee.List([_params['datetimeName'], 'timestamp']))
    
        imgProps = ee.Dictionary(img.toDictionary(propsFrom).rename(propsFrom, propsTo))
    
    
    
        #Subset points that intersect the given image.
        fcSub = fc.filterBounds(img.geometry())

        
        #Reduce the image by regions. Then add metadata to each feature.
        return img.reduceRegions(
            collection= fcSub,
            reducer= _params['reducer'],
            scale= _params['scale'],
            crs= _params['crs']
            ).map(lambda f:
                  f.set(imgProps))
            

        
    results = ic.map(reducing).flatten().filter(ee.Filter.notNull(_params['bandsRename']))    

    
    return results


#Zonal stats fxn StdDev
def zonalStatsStdDev(ic, fc, params):
#Initialize internal params dictionary.
    _params = {
        'reducer': ee.Reducer.stdDev(),
        'scale': None,
        'crs': None,
        'bands': None,
        'bandsRename': None,
        'imgProps': None,
        'imgPropsRename': None,
        'datetimeName': 'datetime',
        'datetimeFormat': 'YYYY-MM-dd HH:mm:ss'
    }

  #Replace initialized params with provided params.
    if params:
        for param in params:
            _params[param] = params[param] or _params[param]

    #print(params)

  #Set default parameters based on an image representative.
    imgRep = ic.first()
    nonSystemImgProps = ee.Feature(None) \
        .copyProperties(imgRep).propertyNames()

    if not _params['bands']: \
     _params['bands'] = imgRep.bandNames() \

    if not _params['bandsRename']: \
      _params['bandsRename'] = _params['bands'] \

    if not _params['imgProps']: \
       _params['imgProps'] = nonSystemImgProps \

    if not _params['imgPropsRename']: \
       _params['imgPropsRename'] = _params['imgProps']
  
    def reducing(img):
    #Select bands (optionally rename), set a datetime & timestamp property.
        img = ee.Image(img.select(_params['bands'], _params['bandsRename'])) \
            .set(_params['datetimeName'], img.date().format(_params['datetimeFormat'])) \
            .set('timestamp', img.get('system:time_start'))

    #Define final image property dictionary to set in output features.
        propsFrom = ee.List(_params['imgProps']) \
            .cat(ee.List([_params['datetimeName'], 'timestamp']))

        propsTo = ee.List(_params['imgPropsRename']) \
            .cat(ee.List([_params['datetimeName'], 'timestamp']))
    
        imgProps = ee.Dictionary(img.toDictionary(propsFrom).rename(propsFrom, propsTo))
    
    
    
        #Subset points that intersect the given image.
        fcSub = fc.filterBounds(img.geometry())

        
        #Reduce the image by regions. Then add metadata to each feature.
        return img.reduceRegions(
            collection= fcSub,
            reducer= _params['reducer'],
            scale= _params['scale'],
            crs= _params['crs']
            ).map(lambda f:
                  f.set(imgProps))
            

        
    results = ic.map(reducing).flatten().filter(ee.Filter.notNull(_params['bandsRename']))    

    
    return results


In [13]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

##############################################################################
# %% import
##############################################################################

#to reset global variables
# for element in dir():
#     if element[0:2] != "__":
#         del globals()[element]
# del element

# print(dir())

#import ee
#from GEE_ImageFusion_Fxns.core_functions import *

#from GEE_ImageFusion_Fxns.prep_functions import *

#ee.Initialize()

##############################################################################
# %% GLOBALS
##############################################################################

    
# region of interest
# location for prediction (outside scene overlap areas)
region = ee.Geometry.Point([-87.03623668,40.41181908]).buffer(10000)
site = ee.Geometry.Point([-87.03623668,40.41181908]).buffer(10000)
#region = ee.Geometry.Point([-118.12627784505537, 44.66875751833964], [-90.803, 38.907])
#path = ee.Number(24)
#row = ee.Number(33)  #target Landsat scene (e.g.: 93/84)

# # define training data temporal bounds broadly
startDate = '2022-06-10'
endDate = '2022-07-30'
startDate_ls7 = ee.String('1999-05-28')
endDate_ls7 = ee.String('2003-05-31'); #End date of ls7 full image

# define training data temporal bounds broadly
# startDate = '2017-03-01'
# endDate = '2018-01-01'
# common bands between sensors that will be used for fusion
#   would need to add functions for other indices (evi etc.).
#   ndvi is exported as 16 bit int so would have to make sure not to rescale
#   reflectance bands if that is what you are planning to predict (line 206)
#   some resturcturing of this code would be necessary if the goal was to
#   predict a multiband image but the core functions should work for any number
#   of bands
commonBandNames = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])

# image collections to use in fusion
# NOTE: if using older Landsat and not using NDVI one would have to modify the
# get_paired_collections script because this script harmonizes NDVI from
# L5 & L7 to L8 based on Roy et al. 2016 (see etmToOli and getPaired functions)

modisCollection = ee.ImageCollection('MODIS/006/MCD43A4')    

ls9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
ls8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
ls89 = ee.ImageCollection(ls9.merge(ls8))
landsatCollection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
ls7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterDate(startDate_ls7, endDate_ls7)
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
ls57 = ee.ImageCollection(ls7.merge(ls5))

modis = ee.ImageCollection('MODIS/006/MCD43A4')

s2_sr = ee.ImageCollection('COPERNICUS/S2_SR')
s2_cloudless = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50
s2_sr_cloudless = get_s2_sr_cld_col(region, startDate, endDate, CLOUD_FILTER)


gsw = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
reach = ee.FeatureCollection('WWF/HydroSHEDS/v1/FreeFlowingRivers')
huc5Sites = ee.FeatureCollection("users/friedmannE/aquaSites_huc05")

huc5Sites_buff = huc5Sites.map(lambda pt: pt.buffer(1000))
#print(huc5Sites_buff.limit(1).getInfo())
#print(site.getInfo())

# landsat band names including qc band for masking
bandNamesLandsat = ee.List(['blue', 'green', 'red',
                            'nir', 'swir1', 'swir2', 'QA_PIXEL'])
landsatBands = ee.List([1, 2, 3, 4, 5, 6, 11])
landsatBandsStr = ee.List(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'])

landsatBands57 = ee.List(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'])
landsatBands89 = ee.List(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'])

#Sentinel Band Names
S2Bands = ['B2','B3','B4','B5','B6','B7', 'QA60']
bandNamesS2 = ['blue','green','red','nir','swir1','swir2', 'QA60']

# modis band names
bandNamesModis = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])
modisBands = ee.List([2, 3, 0, 1, 5, 6])

# radius of moving window
# Note: Generally, larger windows are better but as the window size increases,
# so does the memory requirement and we quickly will surpass the memory
# capacity of a single node (in testing 13 was max size for single band, and
# 10 was max size for up to 6 bands)
kernelRadius = ee.Number(10)
kernel = ee.Kernel.square(kernelRadius)
numPixels = kernelRadius.add(kernelRadius.add(1)).pow(2)

# number of land cover classes in scene
coverClasses = 7

# to export the images to an asset we need the path to the assets folder
path = 'users/friedmannE/FusionTests/'
#scene_name = 'Fusion_python2_2232_fmask89_ls89_noWMask_'
print('This path is:', path)

This path is: users/friedmannE/FusionTests/


In [17]:
    

##############################################################################
# %% get filtered collections
##############################################################################

#landsatCollection = getLandsat(startDate, endDate, country, ls8, region)

# sorted, filtered, paired image retrieval
sortedImages, valImgs = getPaired(startDate, endDate, ls89, ls57, s2_sr_cloudless,
              landsatCollection, landsatBands, bandNamesLandsat,
              S2Bands, bandNamesS2,
              modisCollection, modisBands, bandNamesModis,
              commonBandNames,
              region)

#print(type(sortedImages))
#print(type(fineDup))
#print(type(paired[0]))
#print(sortedImages[0].size().getInfo())
#print(sortedImages[1].size().getInfo())
#print(sortedImages[2].size().getInfo())
subs = makeSubcollections(sortedImages)

subs_meta = subs.getInfo()
#print(subs.get(0).getInfo())
#print(subs.get(1).getInfo())
#print(subs.get(2).getInfo())


##############################################################################
# %% Predict Images
##############################################################################

# loop through each list of paired images
num_lists = subs.length().getInfo()


for i in range(0, num_lists):
    # determine the number of modis images between pairs
    num_imgs = ee.List(ee.List(subs.get(i)).get(2)).length()
    print('# of Imgs', num_imgs.getInfo())
    # determine the remainder of images, if not groups of 10
    remaining = num_imgs.mod(10)
    #print('remaining', remaining.getInfo())
    # create sequence of starting indices for modis images
    index_seq = ee.List.sequence(0, num_imgs.subtract(remaining), 10)

    #print('Prediction groups', index_seq.getInfo())
    # images to be grouped and predicted
    subList = ee.List(ee.List(subs.get(i)).get(2))

    # loop through indices predicting in batches of 10
    for x in range(0, index_seq.length().getInfo()):
        # starting index
        start = ee.Number(index_seq.get(x))

        # ending index
        end = ee.Algorithms.If(start.add(10).gt(num_imgs),
                               num_imgs,
                               start.add(10))

        # group of images to predict
        pred_group = subList.slice(start, end)
        #print(pred_group.getInfo())
        landsat_t01 = ee.List(ee.List(subs.get(i)).get(0))
        #print(landsat_t01.getInfo())
        modis_t01 = ee.List(ee.List(subs.get(i)).get(1))
        #print(landsat_t01.getInfo())
        modis_tp = pred_group
        #print(modis_tp.get(0).getInfo())

        # get the start and end day values and year for the group to use
        # to label the file when exported to asset
        startDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                   .first()
                                   .get('DOY'))
        endDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                 .sort('system:time_start', False)
                                 .first()
                                 .get('DOY'))
        
        #print('endDay', endDay.getInfo())

        year = ee.Date(ee.ImageCollection(pred_group)
                       .sort('system:time_start', False)
                       .first()
                       .get('system:time_start')).format('Y')

        # start and end day of year
        doys = landsat_t01 \
            .map(lambda img: ee.String(ee.Image(img).get('DOY')).cat('_'))



        # register images
        landsat_t01, modis_t01, modis_tp = registerImages(landsat_t01,
                                                          modis_t01,
                                                          modis_tp)
        #print('landsat t01_1', ee.ImageCollection(landsat_t01).first().geometry().getInfo())
        #print('modis t01_1', ee.ImageCollection(modis_t01).first().geometry().getInfo())
        #print('modis_tp_1', ee.ImageCollection(modis_tp).first().geometry().getInfo())

        # prep landsat imagery (mask and format)
        maskedLandsat, pixPositions, pixBN = prepLandsat(landsat_t01,
                                                         kernel,
                                                         numPixels,
                                                         commonBandNames,
                                                         doys,
                                                         coverClasses)

        #print('masked landsat', ee.ImageCollection(maskedLandsat).first().geometry().getInfo())


        # prep modis imagery (mask and format)
        modSorted_t01, modSorted_tp = prepMODIS(modis_t01, modis_tp, kernel,
                                                numPixels, commonBandNames,
                                                pixBN)

        #print('modSorted t01_1', ee.ImageCollection(modSorted_t01).first().geometry().getInfo())



        # calculate spectral distance
        specDist = calcSpecDist(maskedLandsat, modSorted_t01,
                                numPixels, pixPositions)

        # calculate spatial distance
        spatDist = calcSpatDist(pixPositions)

        # calculate weights from the spatial and spectral distances
        weights = calcWeight(spatDist, specDist)

        # calculate the conversion coefficients
        coeffs = calcConversionCoeff(maskedLandsat, modSorted_t01,
                                     doys, numPixels, commonBandNames)



        # predict all modis images in modis tp collection
        prediction = modSorted_tp \
            .map(lambda image:
                 predictLandsat(landsat_t01, modSorted_t01,
                                doys, ee.List(image),
                                weights, coeffs,
                                commonBandNames, numPixels)) \

        #print(type(prediction))
        #print(prediction.length().getInfo())
        #print('prediction', prediction.get(0).getInfo())


        #Image collection of the group of predicted images
        predsColl = ee.ImageCollection(prediction)
        
        #print(prediction.get(0).getInfo())

        ###START EF CODE HERE

        #mask the image collection to keep only water pixels
        water = gsw.select('occurrence').gte(40);
        waterMask = water.selfMask()

        #mask and scale over the image collection
        predsColl = predsColl.map(lambda image: image.clip(region)\
                        .updateMask(waterMask))\
                        .map(applyScaleFactors)
        #print(predsColl.first().getInfo())
        
        #print('watermask geo check', predsColl.first().geometry().centroid().getInfo())
                #First try:
        # calculate the standard deviation for each band within image
#         mean =prediction.map(lambda image: ee.Image(image).reduceRegion(reducer=ee.Reducer.mean(),
#                                               bestEffort=True,
#                                               maxPixels=ee.Number(1e6)))
#         mean = ee.Dictionary(mean).toImage()

#         # calculate the standard deviation for each band within image
#         stddev =prediction.map(lambda image: ee.Image(image).reduceRegion(reducer=ee.Reducer.stdDev(),
#                                               bestEffort=True,
#                                               maxPixels=ee.Number(1e6)))
#         stddev = ee.Dictionary(stddev).toImage()

#         #Second try: reduce by neighborhood
        
#         reducer = ee.Reducer.mean().combine(
#             reducer2 = ee.Reducer.stdDev(),
#             sharedInputs = True)
        
#         neighCol = predsColl.map(lambda image:
#             image.reduceNeighborhood(
#                 reducer = ee.Reducer.mean(),
#                 kernel = ee.Kernel.square(
#                     radius = 1, 
#                     units =  'meters', 
#                     normalize = False)
#                 ).copyProperties(image, ['system:time_start', 'DOY', 'system:footprint', 'system:index']))
        

        #Define parameters for the zonalStats function.
        paramsMean = {
            'reducer': ee.Reducer.mean(),
            'scale': ee.Number(30),
            'crs': predsColl.first().projection(),
            'bands': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'], #['blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'swir1_mean', 'swir2_mean'], #['blue_stdDev', 'green_stdDev', 'red_stdDev', 'nir_stdDev', 'swir1_stdDev', 'swir2_stdDev'],
            'bandsRename': ['blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'swir1_mean', 'swir2_mean'], #['ls_blue', 'ls_green', 'ls_red', 'ls_nir', 'ls_swir1', 'ls_swir2'],
            'imgProps': ['DOY'], #'LANDSAT_SCENE_ID', 'SPACECRAFT_ID'
            'imgPropsRename': ['DOY'], #'scene_id', 'satellite', 
            'datetimeName': 'date',
            'datetimeFormat': 'YYYY-MM-dd'
        }
        
        paramsStdDev = {
            'reducer': ee.Reducer.stdDev(),
            'scale': ee.Number(30),
            'crs': predsColl.first().projection(),
            'bands': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'],
            'bandsRename': ['blue_stdDev', 'green_stdDev', 'red_stdDev', 'nir_stdDev', 'swir1_stdDev', 'swir2_stdDev'],
            'imgProps': ['DOY'], 
            'imgPropsRename': ['DOY'], 
            'datetimeName': 'date',
            'datetimeFormat': 'YYYY-MM-dd'
        }


        ##Extract zonal statistics per point per image.
        LMeanStats = zonalStatsMean(predsColl, huc5Sites_buff, paramsMean)
        LStdDevStats = zonalStatsStdDev(predsColl, huc5Sites_buff, paramsStdDev)#.select(['blue_stdDev', 'green_stdDev', 'red_stdDev', 'nir_stdDev', 'swir1_stdDev', 'swir2_stdDev'])

        
        
        #OPTION 1 (stdDev is too large for some reason for more than 2 days)
        #Mean Zonal stats to pandas Dataframe
#         column_mean = ['system:index','DOY','SiteID','blue_mean','date','green_mean','lat','long','nir_mean','path',\
#                      'red_mean','row','swir1_mean','swir2_mean','timestamp','.geo']
#         nested_listM = LMeanStats.reduceColumns(ee.Reducer.toList(len(column_mean)), column_mean).values().get(0)
#         dataM = nested_listM.getInfo()
#         #print(dataM)
        
#         df_M = pd.DataFrame(dataM, columns=column_mean)
#         print(df_M)
        
#         #StdDev Zonal Stats to pandas Dataframe
#         column_stdDev =  ['blue_stdDev', 'green_stdDev', 'red_stdDev', 'nir_stdDev', 'swir1_stdDev', 'swir2_stdDev']#['system:index','DOY','SiteID','blue_stdDev','date','green_stdDev','lat','long','nir_stdDev','path',\
#              #'red_stdDev','row','swir1_stdDev','swir2_stdDev','timestamp','.geo'] #['blue_stdDev', 'green_stdDev', 'red_stdDev', 'nir_stdDev', 'swir1_stdDev', 'swir2_stdDev']
#         nested_listS = LStdDevStats.reduceColumns(ee.Reducer.toList(len(column_stdDev)), column_stdDev).values().get(0)
#         dataS = nested_listS.getInfo()
#         #print(dataS)
        
#         df_stdDev = pd.DataFrame(dataS, columns=column_stdDev)
#         print(df_stdDev)
        
        #OPTION 2 using GEEMAP
        
        #df_mean = geemap.ee_to_pandas(LMeanStats, selectors=['system:index','DOY','SiteID','blue_mean','date',\
        #                                                     'green_mean','lat','long','nir_mean','path','red_mean',\
        #                                                     'row','swir1_mean','swir2_mean','timestamp','.geo'])
        #print(df_mean)
        
        #df_stdDev = geemap.ee_to_pandas(LStdDevStats, selectors=['system:index','DOY','SiteID','blue_stdDev', 'date',\
        #                                                         'green_stdDev', 'lat','long','red_stdDev',\
        #                                                         'nir_stdDev', 'path','row','swir1_stdDev', 'swir2_stdDev'])
        #print(df_stdDev)
        
        ###END EF CODE HERE
        
        
        #TO EXPORT IMAGE:
        # create a list of new band names to apply to the multiband image
        # NOTE: cant export with names starting with 0
        preds = predsColl.toBands() #ee.ImageCollection(prediction)
        dates = modis_tp.map(lambda img:
                             ee.Image(img).get('system:time_start'))
        predNames = ee.List.sequence(0, prediction.length().subtract(1)) \
            .map(lambda i:
                 commonBandNames\
                     .map(lambda name:
                          ee.String(name)
                          .cat(ee.String(ee.Number(dates.get(i)).format()))))\
            .flatten()
        scene_name = 'Fusion_2232_stdDev_TEST2_'
        
        #export all predictions as a single multiband image
        #each band name corresponds to the timestamp for the image
        #ee.Geometry.Point([-89.041737,37.2031076]).buffer(5000), OR ee.Image(prediction.get(0)).geometry() 
        task = ee.batch.Export.image.toAsset(
            image=preds.rename(predNames).multiply(10000).toInt8().clip(region),
            description= ee.String(scene_name)
                        .cat(year)
                        .cat('_')
                        .cat(startDay.format())
                        .cat('_').cat(endDay.format()).getInfo(),
            assetId=ee.String(path)
                    .cat(ee.String(scene_name))
                    .cat(year)
                    .cat('_')
                    .cat(startDay.format())
                    .cat('_')
                    .cat(endDay.format()).getInfo(),
            region= ee.Image(prediction.get(0)).geometry(),
            scale=30)
        
        #TO EXPORT TABLE
        task2 = ee.batch.Export.table.toDrive(
        collection = LStdDevStats,
        folder = 'FusionImages',
        description = ee.String(scene_name)
                     .cat(year)
                     .cat('_')
                     .cat(startDay.format())
                     .cat('_').cat(endDay.format()).getInfo(),
        fileFormat = 'CSV'
        )

        #task.start()
        #task2.start()
        
        #2456 Aquasat sites with >5 TSS measurements. 1517 with > 10, 1103 with >15, 294 > 50

ORIG Fine # images 19
ORIG MODIS # images 47
['2022-06-14', '2022-06-16', '2022-06-18', '2022-06-21', '2022-06-24', '2022-06-29', '2022-07-01', '2022-07-04', '2022-07-09', '2022-07-12', '2022-07-14', '2022-07-16', '2022-07-19', '2022-07-28']
[['2022-06-14', '2022-06-16', '2022-06-18', '2022-06-21', '2022-06-24', '2022-06-29', '2022-07-01', '2022-07-04', '2022-07-09', '2022-07-12', '2022-07-14', '2022-07-16', '2022-07-19', '2022-07-28'], ['2022-06-14', '2022-06-16', '2022-06-18', '2022-06-21', '2022-06-24', '2022-06-29', '2022-07-01', '2022-07-04', '2022-07-09', '2022-07-12', '2022-07-14', '2022-07-16', '2022-07-19', '2022-07-28'], ['2022-06-14', '2022-06-16', '2022-06-18', '2022-06-21', '2022-06-24', '2022-06-29', '2022-07-01', '2022-07-04', '2022-07-09', '2022-07-12', '2022-07-14', '2022-07-16', '2022-07-19', '2022-07-28'], ['2022-06-14', '2022-06-16', '2022-06-18', '2022-06-21', '2022-06-24', '2022-06-29', '2022-07-01', '2022-07-04', '2022-07-09', '2022-07-12', '2022-07-14', '2022-07

KeyboardInterrupt: 

In [28]:
test = huc5Sites.map(masterFxn)
print(test.first().getInfo())

EEException: A mapped function's arguments cannot be used in client-side operations

In [13]:


#mask the image collection to keep only water pixels
water = gsw.select('occurrence').gte(40);
waterMask = water.selfMask()

#mask over the image collection
predsColl2 = predsColl.map(lambda image: 
                          image.updateMask(waterMask).toInt16()) 

land512M = land512.updateMask(waterMask)


c = land512M.reduceRegion(
    reducer = ee.Reducer.count(),
    geometry = land512.geometry())

#print(c.getInfo())




#to feature collection
print(land512.geometry().getInfo())

predsFeat = ee.FeatureCollection(prediction)


predsColl3 = ee.ImageCollection.fromImages(prediction)
print(type(predsColl3))
print(predsColl3.geometry().getInfo())

#transfomation 

#print(predsColl.first().geometry().transform(maxError=0.1).coordinates().getInfo())
#print(predsColl.first().geometry().transform(maxError = 1).getInfo())
#print(ee.ImageCollection(landsat_t01).first().geometry().getInfo())

{'type': 'Polygon', 'coordinates': [[[-89.5148031831702, 39.40805745626556], [-89.49507683532082, 39.467993857584815], [-89.46507652870619, 39.55899811043739], [-89.73648382545628, 39.61208505217788], [-91.20809557109149, 39.88765040770836], [-91.57921302358913, 39.95388175875285], [-91.5810976619187, 39.95390489556777], [-91.83577522064128, 39.09722809413707], [-92.08532844940895, 38.235552357182385], [-91.54116575138737, 38.13578229157816], [-90.1811037458833, 37.87421938043654], [-90.02049175588384, 37.84217742848612], [-90.01896904746043, 37.84642605281802], [-89.9824308853401, 37.961455538832105], [-89.94039259199953, 38.093566231496055], [-89.85947944610278, 38.34665018100271], [-89.78258951925945, 38.585767307130965], [-89.72884251120058, 38.752177453334156], [-89.69365197802523, 38.86072741272203], [-89.65948652084558, 38.96587247661194], [-89.59616666159732, 39.159964028347055], [-89.5148031831702, 39.40805745626556]]]}
<class 'ee.imagecollection.ImageCollection'>
{'geodesic':

In [ ]:
##############################################################################
# %% Export Images
##############################################################################
        
        
#         task = ee.batch.Export.table.toDrive(
#             collection = LandsatStats,
#             folder = '/content/gdrive/MyDrive/SNiP/RS_fusion/Nietupski_etal_code/GEE-Image-Fusion-Bands-EF',
#             description = ee.String(scene_name)
#                          .cat(year)
#                          .cat('_')
#                          .cat(startDay.format())
#                          .cat('_').cat(endDay.format()).getInfo(),
#             fileFormat = 'CSV'
#             )

        #export all predictions as a single multiband image
        #each band name corresponds to the timestamp for the image
#         task = ee.batch.Export.image.toAsset(
#             image=preds.rename(predNames).toInt16(),
#             description=ee.String(scene_name)
#                         .cat(year)
#                         .cat('_')
#                         .cat(startDay.format())
#                         .cat('_').cat(endDay.format()).getInfo(),
#             assetId=ee.String(path)
#                     .cat(ee.String(scene_name))
#                     .cat(year)
#                     .cat('_')
#                     .cat(startDay.format())
#                     .cat('_')
#                     .cat(endDay.format()).getInfo(),
#             region=ee.Image(prediction.get(0)).geometry(),
#             scale=30)


        
        
#         task = ee.batch.Export.image.toDrive(
#             image = land512,
#             folder = '/content/gdrive/MyDrive/SNiP/RS_fusion/Nietupski_etal_code/GEE-Image-Fusion-Bands-EF',
#             description = ee.String(scene_name)
#                          .cat(year)
#                          .cat('_')
#                          .cat(startDay.format())
#                          .cat('_').cat(endDay.format()).getInfo(),
#             region = ee.Image(land512).geometry(),
#             scale = 30,
#             crs = ee.Image(land512).projection(),
#             maxPixels = 1e13
#             )
        
        #task.start()

In [186]:
#comparing landsat to fused image

##get description of metadata
#print(ee.Algorithms.Describe(predsColl).getInfo())
#print(ee.Algorithms.Describe(landsatRawCol).getInfo())

##Find number of points in the image. They are the same
#print(predsColl.getRegion(predsColl.first().geometry(), 30).getInfo())
#ImageCollection.getRegion: Too many values: 75604334 points x 6 bands x 1 images > 1048576
#print(landsatRawCol.getRegion(predsColl.first().geometry(), 30).getInfo())
#EEException: ImageCollection.getRegion: Too many values: 75604334 points x 6 bands x 2 images > 1048576.


##Try a reduce regions without the fancy function. still doesnt work. But does work for one region on one image for predsColl

reduced = predsColl3.first().reduceRegions(
    collection= reachBounds,
    reducer= ee.Reducer.mean(),
    scale=ee.Number(30),
    crs= predsColl.first().projection())

print(reduced.limit(1).getInfo())

# landArr = landsatRawCol.first().toArray()
# print(type(landArr))
# print(landArr..getInfo()getInfo())

# predsArr = predsColl.first().toArray()
# print(type(landArr))
# print(predsArr.getInfo())


EEException: User memory limit exceeded.

In [367]:
# Extract an image collection
#print(type(predsColl))
#print(predsColl.first().date().format('YYYY-MM-dd').getInfo())

image1 = predsColl.first()
print("fused", image1.geometry().getInfo())
#print(image1.getInfo())



print("Raw", land512.geometry().getInfo())
print("Raw projection", land512.projection().getInfo())
#precision = reachBounds.map(lambda f: f.transform(predsColl.first().projection(), 1))
#print(precision.limit(1).getInfo())

fused {'geodesic': False, 'crs': {'type': 'name', 'properties': {'name': 'EPSG:32615'}}, 'type': 'Polygon', 'coordinates': [[[578085, 4191885], [805815, 4191885], [805815, 4424115], [578085, 4424115], [578085, 4191885]]]}
Raw {'type': 'Polygon', 'coordinates': [[[-89.5148031831702, 39.40805745626556], [-89.49507683532082, 39.467993857584815], [-89.46507652870619, 39.55899811043739], [-89.73648382545628, 39.61208505217788], [-91.20809557109149, 39.88765040770836], [-91.57921302358913, 39.95388175875285], [-91.5810976619187, 39.95390489556777], [-91.83577522064128, 39.09722809413707], [-92.08532844940895, 38.235552357182385], [-91.54116575138737, 38.13578229157816], [-90.1811037458833, 37.87421938043654], [-90.02049175588384, 37.84217742848612], [-90.01896904746043, 37.84642605281802], [-89.9824308853401, 37.961455538832105], [-89.94039259199953, 38.093566231496055], [-89.85947944610278, 38.34665018100271], [-89.78258951925945, 38.585767307130965], [-89.72884251120058, 38.752177453334156

In [10]:
#Landsat image at 5/12/21
landsatRaw = ee.ImageCollection(landsatCollection) \
                    .filterDate(startDate, endDate) \
                    .filterBounds(region) \
                    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
                    .select(landsatBandsStr, bandNamesLandsat) \
                    .map(maskLandsat) \
                    .filterMetadata('CloudSnowMaskedPercent', 'less_than', 20)\
                    .map(lambda image: image \
                         .setMulti({
                             'system:time_start':
                                 ee.Date(image.date().format('y-M-d')) \
                                 .millis(),
                             'DOY': image.date().format('D')
                             })) \
                    .select(commonBandNames) 
land512 = landsatRaw.first()
#print(land512.getInfo())


In [248]:
#mask the image collection to keep only water pixels
water = gsw.select('occurrence').gte(40);
waterMask = water.selfMask()

# mask over the image collection
landsatRawCol = landsatRaw.map(lambda image: 
                               image.updateMask(waterMask).toInt16())
#print(landsatRawCol.first().getInfo())
    
        
#Get reach bounding boxes
# filter to scene of interest generated from above //landsatCol.first().geometry()
filtered = reach.filterBounds(landsatRawCol.first().geometry()).select('REACH_ID', 'LENGTH_KM', 'DOR', 'FLD', 'CSI_FF2', 'ERO_YLD_TO', 'SED')
#apply fxn
reachBounds = filtered.map(getBounds)

print(type(reachBounds))
print(reachBounds.size().getInfo())

#Define parameters for the zonalStats function MEAN.
# params = {
#     'reducer': ee.Reducer.mean(),
#     'scale': ee.Number(30),
#     'crs': landsatRawCol.first().projection(),
#     'bands': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'],
#     'bandsRename': ['ls_blue', 'ls_green', 'ls_red', 'ls_nir', 'ls_swir1', 'ls_swir2'],
#     'imgProps': ['DOY'], #'LANDSAT_SCENE_ID', 'SPACECRAFT_ID'
#     'imgPropsRename': ['DOY'], #'scene_id', 'satellite', 
#     'datetimeName': 'date',
#     'datetimeFormat': 'YYYY-MM-dd'
# }

params = {
    'reducer': ee.Reducer.stdDev(),
    'scale': ee.Number(30),
    'crs': landsatRawCol.first().projection(),
    'bands': ['blue', 'green', 'red', 'nir', 'swir1', 'swir2'],
    'bandsRename': ['blue_stdDev', 'green_stdDev', 'red_stdDev', 'nir_stdDev', 'swir1_stdDev', 'swir2_stdDev'],
    'imgProps': ['DOY'], #'LANDSAT_SCENE_ID', 'SPACECRAFT_ID'
    'imgPropsRename': ['DOY'], #'scene_id', 'satellite', 
    'datetimeName': 'date',
    'datetimeFormat': 'YYYY-MM-dd'
}

#Extract zonal statistics per point per image.
LandsatStats = zonalStats(landsatRawCol, reachBounds, params)
print(type(LandsatStats))
print(LandsatStats.first().propertyNames().getInfo())
print(LandsatStats.size().getInfo())
print(LandsatStats.limit(1).getInfo())

task = ee.batch.Export.table.toDrive(
            collection = LandsatStats,
            folder = '/content/gdrive/MyDrive/SNiP/RS_fusion/Nietupski_etal_code/GEE-Image-Fusion-Bands-EF',
            description = ee.String(scene_name)
                         .cat(year)
                         .cat('_')
                         .cat(startDay.format())
                         .cat('_').cat(endDay.format()).getInfo(),
            fileFormat = 'CSV'
            )
#task.start()

<class 'ee.featurecollection.FeatureCollection'>
1807
<class 'ee.featurecollection.FeatureCollection'>
['date', 'DOY', 'timestamp', 'ls_blue', 'ls_green', 'ls_red', 'ls_nir', 'ls_swir1', 'ls_swir2', 'system:index', 'LENGTH_KM', 'REACH_ID']
2109
{'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-91.58124930892623, 39.89374889374833], [-91.46458138896386, 39.89374889374833], [-91.46458138896386, 39.968750973467344], [-91.58124930892623, 39.968750973467344], [-91.58124930892623, 39.89374889374833]]]}, 'id': 'LC08_024033_20210512_00db000000000000161c', 'properties': {'DOY': '132', 'LENGTH_KM': 16.65, 'REACH_ID': 70571699, 'date': '2021-05-12', 'ls_blue': 8299.079872204473, 'ls_green': 9212.884984025559, 'ls_nir': 8254.507987220448, 'ls_red': 8582.766773162939, 'ls_swir1': 7888.952076677317, 'ls_swir2': 7798.437699680511, 'timestamp': 1620777600000}}]}


In [43]:
print(dir())

for element in dir():

    if element[0:2] != "__":

        del globals()[element]


del element



['In', 'LandsatStats', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i4', '_i40', '_i41', '_i42', '_i43', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'bandNamesLandsat', 'bandNamesModis', 'calcConversionCoeff', 'calcSpatDist', 'calcSpecDist', 'calcWeight', 'coeffs', 'collections', 'commonBandNames', 'country', 'coverClasses', 'doys', 'ee', 'end', 'endDate', 'endDate_ls7', 'endDay', 'etmToOli', 'exit', 'filtered', 'getBounds', 'getDates', 'getPaired', 'get_ipython', 'gsw', 'huc08', 'i', 'index_seq', 'kernel', 'kernelRadius', 'landsatBands', 'landsatBandsStr', 'landsatCollection', 'landsat_t01', 'ls5', 'ls7', 'ls8', 'makeSub